In [ ]:
!pip3 install python-sat

In [2]:
import numpy as np
import time
from random import randrange
from random import seed
from random import shuffle
from pysat.solvers import Glucose4
from pysat.solvers import Minisat22
from pysat.solvers import MapleCM
from pysat.solvers import Lingeling
from pysat.solvers import Minicard
import matplotlib.pyplot as plt
import math
import networkx as nx
import random

## Implementation

In [3]:
def print_grid(grid, m):
    count = 0
    for elem in grid:
        print(elem, end=" ")
        count = (count + 1)%m
        if count==0:
            print()
    print()

class Graph:
    def __init__(self, n):
        self.n = n
        self.adj = [[] for i in range(n)]
        self.degree = 0    #maximum degree

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)
        if len(self.adj[u])>self.degree or len(self.adj[v])>self.degree:
            self.degree+=1

    # Find an arbitrary coloring

    def find_coloring(self, q):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            tmp = [0 for _ in range(q)]
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    tmp[colors[u]] = 1
            for it in range(q):
                if tmp[it] == 0:
                    colors[i] = it
                    break
        return colors

    #find coloring with constraints
    def find_coloring_cons(self, q, border, border_colors):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            blocked = [0 for _ in range(q)]
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    blocked[colors[u]] = 1
            if i in border:
                for clr in range(q):
                    if clr not in border_colors[i]:
                        blocked[clr] = 1
            for it in range(q):
                if blocked[it] == 0:
                    colors[i] = it
                    break
        return colors



    def find_different_coloring(self, q):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            available = [True for _ in range(q)]
            available[0] = False
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    available[colors[u]] = False
            for it in range(q):
                if available[it]:
                    colors[i] = it
                    break
        return colors



    '''
        Forward simulation of the Glauber dynamics;
            coloring: some arbitrary (valid) coloring,
            transitions: a list of (node, color) pairs
    '''

    #Simplify transitions by using bounding lists
    def fix_transitions(self, transitions, q):

        transitions_new = []
        bounding_lists = [set([clr for clr in range(q)]) for _ in range(self.n)]
        for (node, colors) in reversed(transitions):
            #there are some colors that could be blocked or not (maybe_blocked)
            blocked = set()
            available = set([clr for clr in range(q)])
            for u in self.adj[node]:
                if len(bounding_lists[u])==1:
                    for clr in bounding_lists[u]:
                        blocked.add(clr)
                for clr in bounding_lists[u]:
                    available.discard(clr)

            colors_new = []
            for color in colors:
                if color in blocked:
                    continue
                colors_new.append(color)
                if color in available:
                    break

            bounding_lists[node]=set(colors_new)
            transitions_new.append((node,colors_new))

        transitions_new.reverse()
        return transitions_new




    #Each transition is of the form (node, list of (degree+1) colors)
    def glauber_dynamics_perm(self, coloring, transitions):

        for (node, colors) in reversed(transitions):

            for color in colors:
                blocked = False
                for u in self.adj[node]:
                    if (coloring[u] == color):
                        blocked = True

                if not blocked:
                    coloring[node] = color
                    break

        return coloring

    # Return a set 'fam' of vertices with distance at most R from v
    # and a set 'brdr' of all vertices in 'fam' with neighbors outside of 'fam'.

    def family(self, v, R):

        visited = {v}   # Set of visited nodes.
        queue = [v]     # Initialize a queue

        for _ in range(R):
            new_queue = []
            for u in queue:
                for neigh in self.adj[u]:
                    if neigh not in visited:
                        visited.add(neigh)
                        new_queue.append(neigh)
            queue = new_queue.copy()

        border = set()
        for u in queue:
            for neigh in self.adj[u]:
                if neigh not in visited:
                    border.add(u)
                    break

        return visited, border


In [4]:
#checks whether a collapse has occured
def CFTPcollapse_permutation(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(n):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation(graph, q, solver_name):

    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = randrange(n)
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        ret_value = CFTPcollapse_permutation(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")


        print(time.time() - time_before)
        t*=2

    return transitions

In [5]:
#checks whether a collapse has occured for a single vertex
def CFTPcollapse_permutation2(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(1):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr


        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation2(graph, q, solver_name, block):

    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = block[len(transitions)%len(block)]   #systematic scan in block
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)

        ret_value = CFTPcollapse_permutation2(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            #print("Before:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")
            #print("After:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)


        print(time.time() - time_before)
        t*=2

    return transitions

## Experiments

In [ ]:
# Construct an m x m square grid

m = 24

n = m*m

graph = Graph(n)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)

In [6]:
#Construct a periodic m x m grid

m = 24

n = m*m

graph = Graph(n)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)

for j in range(m):
    graph.add_edge(m*j, m*j+m-1)

for i in range(m):
    graph.add_edge(i, m*(m-1) + i)


In [7]:
R = 7
block = []
border = []
for j in range(m):
    for i in range(m):
        if (i<=R or i>=m-R) and (j<=R or j>=m-R):
            block.append(m*j+i)
        if ((i==R+1 or i==m-R-1) and (j<=R or j>=m-R)) or ((i<=R or i>=m-R) and (j==R+1 or j==m-R-1)):
            border.append(m*j+i)

#print(block)
print(len(block))
#print(border)
print(len(border))



225
60


In [ ]:
#total number of edges

sumi = 0
for i in range(n):
    sumi += len(graph.adj[i])
print(sumi//2)

In [ ]:
#add random edges

for i in range(n):
    for j in range(n):
        dist = abs(i%m - j%m) + abs(i//m - j//m)
        a = random.uniform(0, 1)
        if i<j and dist >= 2 and a <= 3.6**(-dist):
            graph.add_edge(i, j)



In [ ]:
R = 8
block, _ = graph.family(0, R)
block = list(block)
print(block)
print(len(block))

In [ ]:
R = 7
block = []
for j in range(m):
    for i in range(m):
        if (i<=R or i>=m-R) and (j<=R or j>=m-R) and (m*j+i==0 or m*j+i>=15):
            block.append(m*j+i)

print(len(block))


In [ ]:
m = 10
n = m*m*m
graph = Graph(n)
G = nx.grid_graph(dim=(m, m, m), periodic = True)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(map[node], map[neigh])
        graph.add_edge(map[node], map[neigh])

In [ ]:
R = 4
block = []
for j in range(m):
    for i in range(m):
        for k in range(m):
            if (i<=R or i>=m-R) and (j<=R or j>=m-R) and (k<=R or k>=m-R):
                block.append(map[(j,i,k)])

print(len(block))


In [ ]:
n = 1000
d = 10
R = 5
graph = Graph(n)
G = nx.random_regular_graph(d, n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(map[node], map[neigh])
        graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block, len(block))


In [ ]:
R = 14
m=40
n=60

G = nx.triangular_lattice_graph(m, n, periodic = True)
n = G.number_of_nodes()
graph = Graph(n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(node, neigh)
        if map[node]>map[neigh]:
            graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block)
print(len(block))

In [ ]:
m=22
n=22
R = 14

G = nx.hexagonal_lattice_graph(m, n, periodic = True)
n = G.number_of_nodes()
graph = Graph(n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(node, neigh)
        graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block, len(block))

In [ ]:
seed(16)

ret_value = 1
transitions = []
t = 1
q = 8
while(True):
    while(len(transitions)<t):
        # Select a random "permutation" transition
        node = block[len(transitions)%len(block)]   #systematic scan
        colors = [clr for clr in range(q)]
        shuffle(colors)
        trans = (node, colors[:(graph.degree+1)])
        transitions.append(trans)


    time_before = time.time()
    print(t)
    coloring1 = graph.find_coloring(q)
    coloring2 = graph.find_different_coloring(q)
    for i in range(n):
        coloring2[i]=1-coloring1[i]
    coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
    coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
    print_grid(coloring1,m)
    print_grid(coloring2,m)
    same = 0
    for i in range(len(coloring1)):
        if (coloring1[i]==coloring2[i]):
            same += 1
    print(same/len(coloring1))
    if coloring1 == coloring2:
        break
    t*=2

In [13]:
q = 7 # Number of colors

seed(16)

solvers = ["glucose"]

lst_x_axis = []
lst_stamps = []

for solver_name in solvers:

    transitions = CFTPsolver_permutation(graph, q, solver_name)

    coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
    coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
    coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
    print(coloring1 == coloring2, "should be True")
    print_grid(coloring1,m)



1
FEASIBLE
False should be False
0.03678607940673828
2
FEASIBLE
False should be False
0.03159451484680176
4
FEASIBLE
False should be False
0.03271007537841797
8
FEASIBLE
False should be False
0.0321958065032959
16
FEASIBLE
False should be False
0.03184771537780762
32
FEASIBLE
False should be False
0.03450131416320801
64
FEASIBLE
False should be False
0.04359579086303711
128
FEASIBLE
False should be False
0.04425764083862305
256
FEASIBLE
False should be False
0.05756688117980957
512
FEASIBLE
False should be False
0.08194661140441895
1024
FEASIBLE
False should be False
0.1344311237335205
2048
FEASIBLE
False should be False
0.24097776412963867
4096
FEASIBLE
False should be False
0.5331413745880127
8192
FEASIBLE
False should be False
1.1359467506408691
16384
FEASIBLE
False should be False
4.9593117237091064
32768
FEASIBLE
False should be False
8.340659618377686
65536
INFEASIBLE
288.2948009967804
True should be True
1 3 4 3 1 6 0 2 4 6 3 0 4 1 2 4 1 4 3 4 0 6 5 3 
3 6 2 1 5 4 2 4 3 4 6 1 3 

In [12]:
for seednum in range(11,12):

    seed(seednum)

    q = 7 # Number of colors

    solvers = ["glucose"]

    lst_x_axis = []
    lst_stamps = []

    for solver_name in solvers:

        transitions = CFTPsolver_permutation2(graph, q, solver_name, block)

        coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
        coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
        coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
        print(coloring1[0] == coloring2[0], "should be True")

1
FEASIBLE
False should be False
0.03484964370727539
2
FEASIBLE
False should be False
0.03349924087524414
4
FEASIBLE
False should be False
0.039511919021606445
8
FEASIBLE
False should be False
0.04242587089538574
16
FEASIBLE
False should be False
0.03328442573547363
32
FEASIBLE
False should be False
0.03690314292907715
64
FEASIBLE
False should be False
0.03586411476135254
128
FEASIBLE
False should be False
0.040274620056152344
256
FEASIBLE
False should be False
0.05131721496582031
512
FEASIBLE
False should be False
0.0841379165649414
1024
FEASIBLE
False should be False
0.13771891593933105
2048
FEASIBLE
False should be False
0.4042520523071289
4096
INFEASIBLE
1.2047169208526611
True should be True
